Training a model means finding the parameters that best fit the training dataset. To do this, we need to be able to measure how well the model fits the data. For linear regression, we find the value of Θ that minimizes the Mean Squared Error (MSE). There are multiple optimization algorithms to do this so we’ll look at a couple.
Ordinary Least Squares
The first method we’re going to code from scratch is called Ordinary Least Squares (OLS). OLS computes the pseudoinverse of X and multiplies it with the target values y. In python this method is pretty easy to implement using scipy.linalg.lstsq() which is the same function that Scikit-Learn’s LinearRegression() class uses.
We’ll try and make ours similar to Scikit-Learn’s library by having fit() and predict() methods. We’ll also give it the attribute coef_ which is the parameters calculated during training.


## Linear Regression

In [1]:
import numpy as np
from scipy.linalg import lstsq

In [2]:
class LinearRegression():
    
    """"
    Class representing a linear regression model.
    
    Models relationship between target variables and attributes by computing line that 
    minimizes the mean squared error.
    
    Parameters
    -------------
    solver : {lstsq}, 
        Optimization method used to minimize the mean squared error in training.
        
        'lstsq' : 
            Ordinary least squares method using scipy.linalg.lstsq
            
    Attributes  
    -------------
    
    coef_ : array of shape (n_features, 1)
        Estimated coefficients for the regression problem.
        
    Notes
    -------------
    This class is capable of being trained using ordinary least sqaures method.
    See solver parameter above.
    
    """
    
    def __init__(self, solver = 'lstsq'):
        self.solver = solver
        
    def fit(self, X, y):
        """
        Fit linear regression model.
        If solver = 'lstsq' model is trained using ordinary least squares.
        
        Parameters
        ------------
        X: array like of shape (n_samples, n_features)
           Training data. Independent variables.
           
        y: array like of shape (n_samples, 1)
           Target values. Dependent variable.
           
        Returns
        -------------
        self: returns an instance of self.
        
        """
        if self.solver == 'lstsq':
            # make sure inputs are numpy arrays
            X = np.array(X)
            y = np.array(y)
            
            # add x_0 = 1 to each instance for the bias term.
            X = np.c_[np.ones((X.shape[0], 1)), X]
            
            # scipy implementation of least squares.
            self.coef_, residues, rank, singular = lstsq(X,y)
            
            return self
        
    def predict(self, X):
        
        """
        Estimate target values using the linear model.
        
        Parameters
        ------------
        X: array-like of shape (n_samples, n_features) Instances.
        
        Returns
        ------------
        C: array of shape (n_samples, 1)
           Estimated targets per instance
        """
        # make sure input are numpy arrays
        X = np.array(X)
        
        # Add x_0 = 1 to each instance for the bias term
        X = np.c_[np.ones((X.shape[0], 1)), X]
        
        return X.dot(self.coef_)
    

To code the fit() method we simply add a bias term to our feature array and perform OLS with the function scipy.linalg.lstsq(). We store the calculated parameter coefficients in our attribute coef_ and then return an instance of self. The predict() method is even simpler. All we do is add a one to each instance for the bias term and then take the dot product of the feature matrix and our calculated parameters.
The ordinary least squares algorithm can get very slow when the number of features grows very large. In these cases it is preferred to use another optimization approach called gradient descent.

Batch Gradient Descent
Gradient descent is a generic optimization algorithm that searches for the optimal solution by making small tweaks to the parameters. To start, you fill Θ with random values (this approach is called random initialization). Then, tweak the parameters until the algorithm converges to a minimum solution by traveling in a direction that decreases the cost function. In our case that means decreasing the MSE.
To travel in the direction that decreases the cost function, you’re going to need to calculate the gradient vector which contains all the partial derivatives of the cost function.

Equation 1. Gradient vector of the cost function
After you calculate the gradient vector you update your parameters in the opposite direction that the gradient vector is pointing. This is where the learning rate (η) comes in to play. The learning rate determines the size of the steps you take in that direction.

Equation 2. Gradient descent step
Batch gradient descent is a version of gradient descent where we calculate the gradient vector of the entire dataset at each step. This means that batch gradient descent does not scale well with very large training sets because it has to load the entire dataset to calculate the next step. If you’re dataset is very large you might want to use stochastic gradient descent or mini-batch gradient descent, but we won’t cover those here. Just know they exist.
Let’s edit our current module to include the option to use batch gradient descent for training. Since we’ll also be creating a Ridge, Lasso, and ElasticNet class, we’ll create a base Regression() class that all of our regressors can inherit from.


In [ ]:
# import numpy as np
# from scipy.linalg import lstsq

from mlscratch.utils.metrics import mean_squared_error


class Regression():
    
    """"
    Class representing our base regression model.
    
    Models relationship between dependent scaler variable y and independent scaler variable X 
    by optimizing a cost function with batch gradient descent.
    
    Parameters
    -------------
    n_iter : float, defualt = 1000
    
        Maximum number of iterations to be used by batch gradient descent.
        
    lr : float, default = 1e-1
    
        Learning rate determining the size of steps in batch gradient descent algorithm.
        
    Attributes  
    -------------
    
    coef_ : array of shape (n_features, 1)
        Estimated coefficients for the regression problem.
        
    Notes
    -------------
    This class is capable of being trained using ordinary least sqaures method.
    See solver parameter above.
    
    """
    
    def __init__(self, n_iter = 1000, lr = 1e-1):
        
        self.n_iter = n_iter
        self.lr = lr
        
    def fit(self, X, y):
        """
        Fit linear regression model with batch gradient descent
        
        
        Parameters
        ------------
        X: array like of shape (n_samples, n_features)
           Training data. Independent variables.
           
        y: array like of shape (n_samples, 1)
           Target values. Dependent variable.
           
        Returns
        -------------
        self: returns an instance of self.
        
        """
         
            # make sure inputs are numpy arrays
        X = np.array(X)
        y = np.array(y)
            
            # add x_0 = 1 to each instance for the bias term.
        X = np.c_[np.ones((X.shape[0], 1)), X]
            
            # store number of samples and features in variables.
        n_samples, n_features = np.shape(X)
        self.training_errors = []
            
            # initialize weights randomly from normal distribution.
        self.coef_ = np.random.randn(n_features, 1)
            
            # batch gradient descent for number of iterations = n_iter
        for _ in range(self.n_iter):
            y_preds = X.dot(self.coef_)
                # calculate mse.
            cost_function = mean_squared_error(y, y_preds)
            self.training_errors.append(cost_function)
                # Gradients of loss function
            gradients = (2/n_samples)*X.T.dot(y_preds - y)
            gradients = gradients
                # update the weights
            self.coef_ = self.lr*gradients
                
        return self
        
    def predict(self, X):
        
        """
        Estimate target values using the linear model.
        
        Parameters
        ------------
        X: array-like of shape (n_samples, n_features) Instances.
        
        Returns
        ------------
        C: array of shape (n_samples, 1)
           Estimated targets per instance
        """
        # make sure input are numpy arrays
        X = np.array(X)
        
        # Add x_0 = 1 to each instance for the bias term
        X = np.c_[np.ones((X.shape[0], 1)), X]
        
        return X.dot(self.coef_)
    

In [6]:
class LinearRegression():
    
    """"
    Class representing a linear regression model.
    
    Models relationship between target variables and attributes by computing line that 
    minimizes the mean squared error.
    
    Parameters
    -------------
    n_iter : float, defualt = 1000
        Maximum number of iterations to be used by batch gradient descent
        
    lr: float, default = 1e-1
        Learning rate determining the size of steps in batch gradient descent.
        
    solver : {lstsq, 'bgd'}, default = 'bgd'
    
        Optimization method used to minimize the mean squared error in training.
        
        
        'lstsq' : 
            Ordinary least squares method using scipy.linalg.lstsq
            
        'bgd' : 
            Batch Gradient Descent.
        
    
    Attributes  
    -------------
    
    coef_ : array of shape (n_features, 1)
        Estimated coefficients for the regression problem.
        
    Notes
    -------------
    This class is capable of being trained using ordinary least sqaures method
    or batch gradient descent. See solver parameter above.
    
    """
    
    def __init__(self, solver = 'lstsq'):
        self.solver = solver
        super(LinearRegression, self).__init__(n_iter=n_iter, lr=lr)
        
    def fit(self, X, y):
        """
        Fit linear regression model.
        If solver = 'lstsq' model is trained using ordinary least squares.
        
        Parameters
        ------------
        X: array like of shape (n_samples, n_features)
           Training data. Independent variables.
           
        y: array like of shape (n_samples, 1)
           Target values. Dependent variable.
           
        Returns
        -------------
        self: returns an instance of self.
        
        """
        if self.solver == 'lstsq':
            # make sure inputs are numpy arrays
            X = np.array(X)
            y = np.array(y)
            
            # add x_0 = 1 to each instance for the bias term.
            X = np.c_[np.ones((X.shape[0], 1)), X]
            
            # scipy implementation of least squares.
            self.coef_, residues, rank, singular = lstsq(X,y)
            
            return self
        # if solver is bgd use batch gradient descent
        
    def predict(self, X):
        
        """
        Estimate target values using the linear model.
        
        Parameters
        ------------
        X: array-like of shape (n_samples, n_features) Instances.
        
        Returns
        ------------
        C: array of shape (n_samples, 1)
           Estimated targets per instance
        """
        # make sure input are numpy arrays
        X = np.array(X)
        
        # Add x_0 = 1 to each instance for the bias term
        X = np.c_[np.ones((X.shape[0], 1)), X]
        return X.dot(self.coef_)
    

Now we have a fully functional linear regression model capable of being trained using batch gradient descent and ordinary least squares. But, what if our data isn’t a straight line? If that’s the case we’ll need a more complex model that can fit nonlinear data like polynomial regression.

## Polynomial Regression

A simple way to model nonlinear data with a linear model is to add powers of each feature as new features, then train the model on this extended set of features. This technique is called polynomial regression.
When you have multiple features, this technique is capable of finding relationships between features because you’re adding all combinations of features up to the given degree.


Ok, now we know polynomial regression is the same as linear regression except we add polynomial features to our dataset before training. Instead of creating a separate PolynomialRegression() class, we’ll add a preprocessing class that can transform your data before training. This way once we build our regularized linear models, they too will be able to perform polynomial regression.
We’ll code it in a similar style to Scikit-Learn’s preprocessing classes. It will have a fit(), transform(), and fit_transform() method.

In [2]:
"""" Module containing classes for preprocessing data."""
from itertools import combinations_with_replacement
import numpy as np
from scipy.special import factorial

class PolynomialFeatures:
    """
    Generate Polynomial Features.
    
    Generate a new matrix of features including all combinations of different
    polynomial features less than or equal to specified degree. No bias term is
    calculated in this implementation because our regression models create a bias
    term when they are trained.
    
    Parameters
    ------------
    
    degree : int, default=2
        Degree of polynomial features to be created
        
    Attributes
    -------------
    n_input_features : int
        The total number of input features.
    n_output_features : int
        The total number of output features computed by iterating over all possible
        polynomial combinations of input features.
        
    """
    
    def __init__(self, degree = 2):
        self.degree = degree
        
    def fit(self, X):
        """
        Compute the number of output featurs.
        
            n_output_features = (n+d)!/d!n!
        where n is the number of input features and d is the degree.
        
        Parameters 
        X: array-like of shape (n_samples, n_features)
            Feature matrix to be transformed into polynomial feature matrix
        
        """
        # Make sure input is numpy array.
        X = np.array(X)
        # Store number of input features in attribute
        self.n_input_features = X.shape[1]
        # calculate numerator and denominator of equation listed above.
        numerator = factorial(self.n_input_features + self.degree)
        denominator = factorial(self.degree)*factorial(self.n_input_features)
        # Calculate number of output features minus 1 to subtract bias term
        self.n_output_features = int(numerator/denominator) - 1
        

def transform(self, X):
    """"
    Transform data to polynomial feature matrix.
    
    Parameters
    ------------
    X : array-like of shape (n_samples, n_features)
            Feature matrix to be transformed into polynomial feature matrix.
            
    Returns
    ------------
    
    X: array like of shape (n_samples, n_output_features)
       Transformed polynomial feature matrix where n_output_features is
       the number of output features after polynomial transformation.
       
       """
    # Generate all combinations of features indices and store them in tuples.
    combos = [combinations_with_replacment(range(self.n_input_features), i)
              for i in range(1, self.degree + 1)]
    # Create a new array of the desired output shape
    X_new = np.empty((X.shape[0], self.n_output_features))
    # Multiply features for each combination tuple in combinations
    for i, index_combos in enumerate(combinations):
        X_new[:, i]=np.prod(X[:, index_combos], axis = 1)
        
    return X_new

def fit_tranform(self, X):
    """
    Compute the number of output features and transform to polynomial feature matrix.
    
    Parameters
    -----------
    
    X: array like of shape (n_samples, n_features)
    Feature matrix to be transformed into polynomial feature matrix.
    
    Returns
    -----------
    X : array-like of shape (n_samples, n_output_features)
        Transformed polynomial feature matrix where n_output_features is
        the number of output features after polynomial transformation.
        
    """
    self.fit(X)
    
    return self.transform(X)


If you try to perform polynomial regression at this point you might get an error during training. This is because you’re using gradient descent as the training algorithm. When we transform our features to add polynomial terms, it is very important we normalize our data if we’re using an iterative training algorithm like gradient descent. If we don’t, we risk encountering exploding gradients.

### Data Normalization

There are many ways to normalize data, but we will stick with one of the simplest. By subtracting out the mean and dividing each instance by the standard deviation for each feature we effectively standardize our data. Meaning all our features are centered with a mean of 0 and unit variance.
Let’s add a class called StandardScaler() to our preprocessing.py module. We’ll include an inverse_transform() method here in case we ever need to return data to its original state after it has been standardized.


In [3]:
class StandardScaler:
    
    """
    Standardize features by centering the mean to 0 and unit variance.
    
    The standard score of an instance is calculated by:
    
    z =  (x-u)/s
    
    where u is the mean of the training data and s is the standard deviation.
    
    Standardizing data is often necessary before training many machine 
    learning models to avoid problems like exploding, vanishing gradients
    and feature dominance.
    
    Attributes
    ------------
    mean_ : numpy array of shape (n_features, )
        The mean of each feature in the training set.
    var_ : numoy array of shape (n_features, )
        The variance of each feature in the training set.
        
    """
    
    def fit(self, X):
        """
        Calculate and store the mean and variance of each feature in the 
        training set.
        
        Parameters
        -----------
        X: array like of shape (n_samples, n_features)
            Data set to calculate mean and variance of.
            
        """
        self.mean_ = np.mean(X, axis = 0)
        self.var_ = np.var(X, axis = 0)
        
    def transform(self, X):
        """
        Standardize data by subtracting out the mean and dividing by
        standard deviation calculated during fitting.
        
        Parameters
        -----------
        
        X: array like of shape (n_samples, n_features)
           Data to standardized.
           
        Returns
        -----------
        
        X_std : array like of shape (n_samples, n_features)
            Standardized data.

SyntaxError: unexpected EOF while parsing (<ipython-input-3-48ad59d36b5f>, line 39)

Link: https://towardsdatascience.com/ml-from-scratch-linear-polynomial-and-regularized-regression-models-725672336076